## Explainer notebook on how to load the data

In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [ ]:
import decoding_analysis_vis as dav
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import scipy
import xarray as xr
sns.set_palette('colorblind')

### Load all sessions using the following command:

In [ ]:
AllSesDict = {xx: dav.AllSessions(memory_efficient=True, sess_type=xx) for xx in ['sens', 'proj']}